In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
path = "2025 ASA DataFest-update2-2025-03-19/"
leases_df = pd.read_csv(path + "Leases.csv")
prices_df = pd.read_csv(path + "Price and Availability Data.csv")
occupancy_df = pd.read_csv(path + "Major Market Occupancy Data-revised.csv")
unemployment_df = pd.read_csv(path + "Unemployment.csv")

cities_to_state = {
    'Atlanta' : 'GA',
    'Austin' : 'TX',
    'Baltimore' : 'MD',
    'Boston' : 'MA',
    'Charlotte' : 'NC',
    'Chicago' : 'IL',
    'Chicago Suburbs' : 'IL',
    'Dallas/Ft Worth' : 'TX',
    'Denver' : 'CO',
    'Detroit' : 'MI',
    'Houston' : 'TX',
    'Los Angeles':'CA',
    'Manhattan':'NY',
    'Nashville': 'TN',
    'Northern New Jersey' : 'NJ',
    'Northern Virginia' : 'VA', 
    'Orange County' : 'CA',
    'Philadelphia' : 'PA',
    'Phoenix' : 'AZ',
    'Raleigh/Durham' : 'NC',
    'Salt Lake City' : 'UT',
    'San Diego' : 'CA',
    'San Francisco' : 'CA',
    'Seattle' : 'WA',
    'South Bay/San Jose' : 'CA',
    'South Florida' : 'FL',
    'Southern Maryland' : 'MD',
    'Tampa' : 'FL',
    'Washington D.C' : 'MD'
}

#deleting leases under 10000sf and the industry is in 
leases_df = leases_df[leases_df.leasedSF > 10000]
leases_df = leases_df[leases_df.internal_industry.isin([
    "Financial Services and Insurance",
    "Technology, Advertising, Media, and Information",
    "Business, Professional, and Consulting Services (except Financial and Legal) - Including Accounting"
])]

leases_df['state'] = leases_df['market'].map(cities_to_state)

RBA = leases_df.dropna(subset=['availability_proportion'])

# Group the data by market, internal_class (O/A), year, and quarter
df_grouped = RBA.groupby(['state', 'market', 'internal_class', 'year', 'quarter']).agg({
    'availability_proportion': 'mean',
    'overall_rent' : 'mean'
}).reset_index()

# Sort the data to ensure proper order for calculating differences
df_grouped = df_grouped.sort_values(by=['state','market', 'internal_class', 'year', 'quarter'])

# Initialize an empty list to store the differences
differences = []

# Iterate over each market and internal_class (O/A)
for (state, market, internal_class), group in df_grouped.groupby(['state','market', 'internal_class']):
    group = group.reset_index(drop=True)  # Reset index for easier iteration
    for i in range(1, len(group)):  # Start from the second row to calculate differences
        diff = group.loc[i, 'availability_proportion'] - group.loc[i - 1, 'availability_proportion']
        differences.append({
            'state' : state,
            'market': market,
            'internal_class': internal_class,
            'year': group.loc[i, 'year'],
            'quarter': group.loc[i, 'quarter'],
            'time_index': f"{group.loc[i, 'year']}-{group.loc[i, 'quarter']}",  # Combine year and quarter
            'difference': diff,
            'availability_proportion': group.loc[i, 'availability_proportion'],
            'overall_rent': group.loc[i, 'overall_rent']

        })

# Convert the differences list into a DataFrame for easier analysis
differences_df = pd.DataFrame(differences)

unemployment_df.rename(columns={'Year': 'year', 'Quarter': 'quarter'}, inplace=True)

unemployment_df = unemployment_df.groupby(['state','year','quarter']).agg({'unemployment_rate':'mean'}).reset_index()


merged = differences_df.merge(
    unemployment_df,  # Select relevant columns
    on=['state', 'year', 'quarter'],  # Merge keys
    how='left'  # Use 'left' join to keep all rows in differences_df
)

print(merged.head())





  state   market internal_class  year quarter time_index  difference  \
0    AZ  Phoenix              A  2018      Q2    2018-Q2    0.016586   
1    AZ  Phoenix              A  2018      Q3    2018-Q3    0.013740   
2    AZ  Phoenix              A  2018      Q4    2018-Q4    0.000354   
3    AZ  Phoenix              A  2019      Q1    2019-Q1   -0.010936   
4    AZ  Phoenix              A  2019      Q2    2019-Q2   -0.006202   

   availability_proportion  overall_rent  unemployment_rate  
0                 0.186043     26.231819           4.700000  
1                 0.199783     26.816695           4.733333  
2                 0.200137     27.361708           4.966667  
3                 0.189200     27.145068           4.966667  
4                 0.182998     27.395518           4.800000  


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

markets = merged['market'].unique()
market_results = {}

for mkt in markets:
    df_mkt = merged[merged['market'] == mkt]
    X = df_mkt[['overall_rent', 'unemployment_rate', 'difference']]
    y = df_mkt['availability_proportion']
    
    # Skip if too few samples
    if len(df_mkt) < 30:
        continue
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    market_results[mkt] = {
        'rmse': rmse,
        'coef': model.coef_,
        'intercept': model.intercept_
    }

# View results
for mkt, res in market_results.items():
    print(f"\nMarket: {mkt}")
    print("  RMSE:", round(res['rmse'], 4))
    print("  Coefficients:", res['coef'])
    print("  Intercept:", res['intercept'])


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_reg


Market: Phoenix
  RMSE: 0.0447
  Coefficients: [ 0.02824733 -0.0094669   1.87370684]
  Intercept: -0.5334954675615395

Market: Los Angeles
  RMSE: 0.0164
  Coefficients: [ 0.01097401 -0.00398607  0.05306663]
  Intercept: -0.23675227725962297

Market: San Diego
  RMSE: 0.0324
  Coefficients: [ 0.00607101 -0.00191371  1.09135748]
  Intercept: -0.028127367596793795

Market: San Francisco
  RMSE: 0.0968
  Coefficients: [-0.01817907 -0.00358585  1.8476849 ]
  Intercept: 1.5508003162842952

Market: South Florida
  RMSE: 0.03
  Coefficients: [ 3.13509453e-03 -6.67307582e-04  1.15074688e+00]
  Intercept: 0.06826656064193841

Market: Tampa
  RMSE: 0.0269
  Coefficients: [ 0.01321292 -0.00164204  0.65861329]
  Intercept: -0.1717196372747589

Market: Atlanta
  RMSE: 0.0563
  Coefficients: [ 0.01655415 -0.00757986  0.82700024]
  Intercept: -0.21941514857844216

Market: Chicago Suburbs
  RMSE: 0.0335
  Coefficients: [ 1.50208984e-02 -7.68485378e-04  1.19391375e+00]
  Intercept: -0.0798585914553332

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_reg

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

markets = merged['market'].unique()
market_results = {}

for mkt in markets:
    df_mkt = merged[merged['market'] == mkt]
    X = df_mkt[['overall_rent', 'unemployment_rate', 'availability_proportion']]
    y = df_mkt['difference']
    
    # Skip if too few samples
    if len(df_mkt) < 30:
        continue
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    market_results[mkt] = {
        'rmse': rmse,
        'coef': model.coef_,
        'intercept': model.intercept_
    }

# View results
for mkt, res in market_results.items():
    print(f"\nMarket: {mkt}")
    print("  RMSE:", round(res['rmse'], 4))
    print("  Coefficients:", res['coef'])
    print("  Intercept:", res['intercept'])



Market: Phoenix
  RMSE: 0.0081
  Coefficients: [-0.00464157  0.00301349  0.14166084]
  Intercept: 0.08803246339066167

Market: Los Angeles
  RMSE: 0.0069
  Coefficients: [0.00102004 0.00124276 0.01099277]
  Intercept: -0.05146259915668853

Market: San Diego
  RMSE: 0.0103
  Coefficients: [-0.00131772  0.0006295   0.11058744]
  Intercept: 0.0281107379647214

Market: San Francisco
  RMSE: 0.0152
  Coefficients: [0.00159818 0.00325219 0.0793185 ]
  Intercept: -0.14100081941898657

Market: South Florida
  RMSE: 0.007
  Coefficients: [-0.00091492  0.0007517   0.10822176]
  Intercept: 0.01289293705930756

Market: Tampa
  RMSE: 0.0069
  Coefficients: [-0.00100514  0.00094377  0.06241458]
  Intercept: 0.013795986558937523

Market: Atlanta
  RMSE: 0.0142
  Coefficients: [0.00144626 0.00667303 0.03481502]
  Intercept: -0.07136989420841543

Market: Chicago Suburbs
  RMSE: 0.011
  Coefficients: [-0.00225043  0.00253269  0.1077226 ]
  Intercept: 0.013490751427078411

Market: Boston
  RMSE: 0.0072


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_reg